In [1]:
!pip install -r requirement.txt -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!nvcc --version

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!python -c "import torch; print(torch.cuda.is_available()); print(torch.version.cuda); print(torch.cuda.get_device_name(0))"

False
11.6


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 329, in get_device_name
    return get_device_properties(device).name
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 359, in get_device_properties
    _lazy_init()  # will define _get_device_properties
  File "c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\torch\cuda\__init__.py", line 217, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


In [4]:
!python -c "from allennlp.modules.feedforward import FeedForward; print('✅ All good')"


✅ All good


In [1]:
!python tokenizer.py

In [2]:
!python layers.py

In [3]:
!python model.py

c:\Users\varun\anaconda3\envs\allennlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# import torch
# from torch.utils.data import DataLoader
# from model import MolGen  # Ensure you are importing your updated model file
# # Assume 'data' is your list of SMILES strings.

# gan_mol = MolGen(data, hidden_dim=128, lr=1e-3, device='cpu')

# # Manually create a DataLoader using the tokenizer's batch_tokenize function.
# loader = DataLoader(
#     data,
#     batch_size=128,
#     shuffle=True,
#     collate_fn=gan_mol.tokenizer.batch_tokenize,  # Use the tokenizer's method directly
#     num_workers=0  # Set to 0 for CPU-only training
# )


In [ ]:
import torch
import os
import csv
from io import StringIO
from model import MolGen, reward_function  # Make sure this import reflects the updated model.py!

# ---------------------
# Embedded CSV dataset as a multiline string.
csv_data = """gdb_129253,N=C1OCC2=C1ON=N2,3.48641,1.68335,1.14344,3.6274,64.64,-0.2747,-0.0814,0.1933,949.6326,0.078874,-468.773686,-468.76723,-468.766286,-468.804646,24.184,-1258.179390342,-1264.793335202,-1271.311271185,-1173.970192578
gdb_129254,N=C1OCC2=CC=NN12,3.36052,1.80963,1.18504,4.5309,71.17,-0.2586,-0.0222,0.2364,955.2363,0.104767,-432.896034,-432.889654,-432.88871,-432.926943,24.581,-1471.287094323,-1479.726462864,-1487.430390857,-1373.9309555
gdb_129255,N=C1OCC2=CN=NN12,3.42804,1.79621,1.18749,5.6683,66.72,-0.2764,-0.0438,0.2326,936.4561,0.092449,-448.925698,-448.919411,-448.918467,-448.956589,23.761,-1341.2973499551,-1348.90589658,-1356.016828568,-1250.307289937
gdb_129256,N=C1OCC2=NC=NN12,3.45064,1.82243,1.20166,3.1844,66.14,-0.2744,-0.0366,0.2378,928.329,0.093208,-448.954255,-448.948043,-448.947099,-448.985067,23.454,-1359.217124468,-1366.872734268,-1373.983666256,-1268.1774912391
gdb_129257,N=C1ON=C(C=C1)C#C,5.67315,1.065,0.89667,3.2179,80.61,-0.2427,-0.0871,0.1556,1198.7634,0.088526,-415.552846,-415.545347,-415.544402,-415.58503,28.231,-1404.880326889,-1411.729587624,-1418.839892103,-1315.317849846
gdb_129258,N=C1ON=C(C=C1)C#N,5.64149,1.07,0.89941,2.794,73.91,-0.2671,-0.1102,0.1569,1169.3214,0.078081,-431.652864,-431.645719,-431.644775,-431.684861,26.224,-1319.038350707,-1325.220569375,-1331.738505358,-1235.735903448
gdb_129259,N=C1ON=C(C=N1)C#C,5.84402,1.08456,0.91479,3.7443,76.59,-0.2595,-0.1101,0.1494,1164.4871,0.076419,-431.59642,-431.588957,-431.588013,-431.629446,27.292,-1283.619232711,-1289.601903517,-1296.1198395,-1200.962492213
gdb_129260,N=C1ON=C(C=N1)C#N,5.81352,1.08851,0.91684,0.6114,69.94,-0.2869,-0.1342,0.1527,1136.3218,0.065891,-447.694242,-447.68712,-447.686176,-447.727117,25.3
"""

# ---------------------
# Parse CSV data and extract SMILES strings.
smiles_list = []
csv_reader = csv.reader(StringIO(csv_data))
for row in csv_reader:
    if len(row) > 1 and row[0].startswith("gdb"):
        smiles_list.append(row[1])

print("Extracted SMILES:", smiles_list)

# ---------------------
# Instantiate the MolGen model on CPU using the extracted SMILES.
gan_mol = MolGen(smiles_list, hidden_dim=128, lr=1e-3, device='cpu')

# Create the DataLoader (set num_workers=0 for CPU-only training).
loader = gan_mol.create_dataloader(smiles_list, batch_size=128, shuffle=True, num_workers=0)

# Train the GAN for 20,000 steps, evaluating every 100 steps.
gan_mol.train_n_steps(loader, max_step=20000, evaluate_every=100)

# Optionally, perform explicit RL training:
# gan_mol.train_rl_n_steps(reward_function, num_steps=2000, batch_size=128, evaluate_every=100)

# ---------------------
# Save trained models.
os.makedirs('saved_models', exist_ok=True)
torch.save(gan_mol.generator.state_dict(), 'saved_models/generator.pth')
torch.save(gan_mol.discriminator.state_dict(), 'saved_models/discriminator.pth')

print("Model saved successfully.")


Extracted SMILES: ['N=C1OCC2=C1ON=N2', 'N=C1OCC2=CC=NN12', 'N=C1OCC2=CN=NN12', 'N=C1OCC2=NC=NN12', 'N=C1ON=C(C=C1)C#C', 'N=C1ON=C(C=C1)C#N', 'N=C1ON=C(C=N1)C#C', 'N=C1ON=C(C=N1)C#N']
Generated examples: ['ON2)==1)CNON', '(NC=)']
GAN step    0: Validity = 0.0600
Generated examples: ['N#CO1=N', 'NNCNN2N#NNNO1ONNC(']
GAN step  100: Validity = 0.1600
Generated examples: ['N#', '=NO12O=)N=2O']
GAN step  200: Validity = 0.1600
Generated examples: ['N2NO2C212', 'NC#']
GAN step  300: Validity = 0.3400
Generated examples: ['NC2CN', 'NCO=)=']
GAN step  400: Validity = 0.3300
Generated examples: ['N21CO1=1=C22)=N#1ON', 'N2=']
GAN step  500: Validity = 0.2600
Generated examples: ['<eos>)2C<eos>NON<eos>2=21C=C2(2', 'C()']
GAN step  600: Validity = 0.0900
Generated examples: ['NC12)C12N==', '11)1N(CN2C1=CC#N']
GAN step  700: Validity = 0.1000
Generated examples: [')(1=', 'NO']
GAN step  800: Validity = 0.0500
Generated examples: ['2', 'N2#2)(']
GAN step  900: Validity = 0.0500
Generated examples: ['

In [10]:
gan_mol.eval()

print('ok')

ok


In [4]:
import torch
from model import MolGen
from rdkit import Chem

# ------------------------------------------------------------------------------
# Set up a representative SMILES list for tokenizer initialization.
data = [
    'N=C1OCC2=C1ON=N2',
    'N=C1OCC2=CC=NN12',
    'N=C1OCC2=CN=NN12',
    'N=C1OCC2=NC=NN12',
    'N=C1ON=C(C=C1)C#C',
    'N=C1ON=C(C=C1)C#N',
    'N=C1ON=C(C=N1)C#C',
    'N=C1ON=C(C=N1)C#N'
]

# Instantiate your model on CPU
gan_mol = MolGen(data, hidden_dim=128, lr=1e-3, device='cpu')

# Load the trained generator weights (ensure that this file exists from training)
state_dict = torch.load("saved_models/generator.pth", map_location=torch.device('cpu'))
gan_mol.generator.load_state_dict(state_dict)

# ------------------------------------------------------------------------------
# Generate 8 SMILES strings using the trained generator.
smiles_list = gan_mol.generate_n(8)

# ------------------------------------------------------------------------------
# Save the generated SMILES strings into a .smi file (one SMILES per line)
output_file = "generated_molecules.smi"
with open(output_file, "w") as f:
    for smi in smiles_list:
        f.write(smi + "\n")

print(f"Generated SMILES have been saved to {output_file}")


Generated SMILES have been saved to generated_molecules.smi
